In [ ]:
import json
import os



In [ ]:
# Load JSON
# Use absolute path to ensure it works regardless of working directory
json_path = '/Users/yashvibhagat/large_exper_unstructured_data/3_combine_jsons_together/combined_papers.json'

with open(json_path, 'r') as f:
    data = json.load(f)

print(f"Successfully loaded data. Number of papers: {len(data.get('papers', []))}")


In [ ]:
# STEP 1: Flatten the nested JSON structure into a list of curves
# The JSON has this structure: papers -> graphs -> curves
# We need to extract all curves and their metadata into a flat list
# IMPORTANT: Capture axis labels WITH units (e.g., "True stress, σ (MPa)" or "Engineering strain (ε)")

curves_list = []

# Loop through all papers
for paper in data.get('papers', []):
    # For each paper, loop through all graphs
    for graph in paper.get('graphs', []):
        # Get axis labels WITH units - these are important for unit conversion later
        x_axis_label_full = graph.get('x_axis_label', '')  # e.g., "Engineering strain (ε)" or "True strain, ε (%)"
        y_axis_label_full = graph.get('y_axis_label', '')  # e.g., "Engineering stress, σ (MPa)" or "True stress, σ (GPa)"
        
        y_unit = 'unknown'  # default
        if '(MPa)' in y_axis_label_full or 'MPa' in y_axis_label_full:
            y_unit = 'MPa'
        elif '(GPa)' in y_axis_label_full or 'GPa' in y_axis_label_full:
            y_unit = 'GPa'
        elif 'MPa' in y_axis_label_full:
            y_unit = 'MPa'
        elif 'GPa' in y_axis_label_full:
            y_unit = 'GPa'
       
        
        # For each graph, loop through all curves
        for curve in graph.get('curves', []):
            # Extract Kocks-Mecking hardening parameters (knocking parameters)
            km_params = curve.get('Kocks–Mecking_hardening_parameters', {})
            
            # Create a dictionary with all information for this curve
            curve_info = {
                'paper_title': paper.get('paper_title', ''),
                'graph_id': graph.get('graph_id', ''),
                'curve_id': curve.get('curve_id', ''),
                'curve_label': curve.get('curve_label', ''),
                'x_axis_label': x_axis_label_full,  # Full label WITH units: "Engineering strain (ε)" or "True strain, ε (%)"
                'y_axis_label': y_axis_label_full,  # Full label WITH units: "Engineering stress, σ (MPa)" or "True stress, σ (GPa)"
                
                'y_unit': y_unit,  # Extracted unit for y-axis (MPa or GPa) - important for conversion!
                'alloy_composition': curve.get('alloy_composition_in_percentages_corresponding_to_this_curve', {}),
                'curve_data': curve.get('curve_raw_data', {}).get('data', []),  # The actual x,y points
                # Kocks-Mecking (Knocking) Parameters
                'km_theta0_MPa': km_params.get('theta0_MPa', None),
                'km_sigma_sat_MPa': km_params.get('sigma_sat_MPa', None),
                'km_fit_strain_range_min': km_params.get('fit_strain_range', [None, None])[0] if km_params.get('fit_strain_range') else None,
                'km_fit_strain_range_max': km_params.get('fit_strain_range', [None, None])[1] if km_params.get('fit_strain_range') else None,
                'km_savgol_window_points': km_params.get('savgol_filter', {}).get('window_points', None),
                'km_savgol_poly_order': km_params.get('savgol_filter', {}).get('poly_order', None),
                'km_goodness_of_fit_R2': km_params.get('goodness_of_fit_R2', None)
               
                
            }
            curves_list.append(curve_info)

print(f"✓ Extracted {len(curves_list)} curves")
print(f"\nSample curve structure:")
print(f"  Keys: {list(curves_list[0].keys())}")
print(f"  Curve ID: {curves_list[0]['curve_id']}")
print(f"  X-axis label (with units): {curves_list[0]['x_axis_label']}")
print(f"  Y-axis label (with units): {curves_list[0]['y_axis_label']}")

print(f"  Y-axis unit: {curves_list[0]['y_unit']}")
print(f"  Number of data points: {len(curves_list[0]['curve_data'])}")
print(f"  Alloy composition: {curves_list[0]['alloy_composition']}")

print(f"\n📊 Kocks-Mecking (Knocking) Parameters:")
print(f"  Theta0 (MPa): {curves_list[0]['km_theta0_MPa']}")
print(f"  Sigma_sat (MPa): {curves_list[0]['km_sigma_sat_MPa']}")
print(f"  Fit strain range: [{curves_list[0]['km_fit_strain_range_min']}, {curves_list[0]['km_fit_strain_range_max']}]")
print(f"  Savgol filter (window, poly): ({curves_list[0]['km_savgol_window_points']}, {curves_list[0]['km_savgol_poly_order']})")
print(f"  R² goodness of fit: {curves_list[0]['km_goodness_of_fit_R2']}")




In [ ]:
# STEP 2: Convert stress values from GPa to MPa for consistency
# This ensures all stress values are in the same unit (MPa) for future processing

converted_count = 0
unchanged_count = 0
unknown_unit_count = 0

for curve_info in curves_list:
    y_unit = curve_info.get('y_unit', 'unknown')
    curve_data = curve_info.get('curve_data', [])
    
    if y_unit == 'GPa':
        # Convert all y-values from GPa to MPa (multiply by 1000)
        for data_point in curve_data:
            if 'y' in data_point:
                data_point['y'] = data_point['y'] * 1000
        
        # Update the unit label
        curve_info['y_unit'] = 'MPa'
        
        # Update the y_axis_label to reflect MPa (replace GPa with MPa)
        original_label = curve_info.get('y_axis_label', '')
        if 'GPa' in original_label:
            curve_info['y_axis_label'] = original_label.replace('GPa', 'MPa')
        
        converted_count += 1
        
    elif y_unit == 'MPa':
        # Already in MPa, no conversion needed
        unchanged_count += 1
        
    else:
        # Unknown unit - might need manual review
        unknown_unit_count += 1

print("="*60)
print(" Unit Conversion")
print("="*60)
print(f"✓ Converted from GPa to MPa: {converted_count} curves")
print(f"✓ Already in MPa (unchanged): {unchanged_count} curves")
print(f"⚠️  Unknown unit (may need review): {unknown_unit_count} curves")
print(f"\nTotal curves processed: {len(curves_list)}")

# # Show sample of converted curve
# if converted_count > 0:
#     # Find first converted curve to show example
#     for curve_info in curves_list:
#         if 'GPa' not in curve_info.get('y_axis_label', '') and curve_info.get('y_unit') == 'MPa':
            # This was likely converted (or already in MPa)
            # sample_data = curve_info.get('curve_data', [])[:3]  # First 3 points
            # print(f"\n📊 Sample curve (y-axis in MPa):")
            # print(f"  Curve ID: {curve_info['curve_id']}")
            # print(f"  Y-axis label: {curve_info['y_axis_label']}")
            # print(f"  Y-axis unit: {curve_info['y_unit']}")
            # print(f"  Sample data points (first 3):")
            # for i, point in enumerate(sample_data):
            #     print(f"    Point {i+1}: x={point['x']:.4f}, y={point['y']:.2f} MPa")
            # break

print("\n✓ All stress values are now standardized to MPa!")



In [ ]:
# STEP 3: Create DataFrame with metadata, units, and alloy composition
# This creates a tabular structure without extracting features from curve_data yet

import pandas as pd

# First, let's prepare the data for DataFrame creation
df_data = []

for curve_info in curves_list:
    row = {
        # Metadata
        'paper_title': curve_info.get('paper_title', ''),
        'graph_id': curve_info.get('graph_id', ''),
        'curve_id': curve_info.get('curve_id', ''),
        'curve_label': curve_info.get('curve_label', ''),
        
        # Axis labels and units
        'x_axis_label': curve_info.get('x_axis_label', ''),
        'y_axis_label': curve_info.get('y_axis_label', ''),
        'y_unit': curve_info.get('y_unit', ''),
        
        
        
        # Kocks-Mecking (Knocking) Parameters
        'km_theta0_MPa': curve_info.get('km_theta0_MPa', None),
        'km_sigma_sat_MPa': curve_info.get('km_sigma_sat_MPa', None),
        'km_fit_strain_range_min': curve_info.get('km_fit_strain_range_min', None),
        'km_fit_strain_range_max': curve_info.get('km_fit_strain_range_max', None),
        'km_savgol_window_points': curve_info.get('km_savgol_window_points', None),
        'km_savgol_poly_order': curve_info.get('km_savgol_poly_order', None),
        'km_goodness_of_fit_R2': curve_info.get('km_goodness_of_fit_R2', None),
       
        
        # Store curve_data as a reference (we'll extract features later)
        'curve_data': curve_info.get('curve_data', [])
    }
    
    # Expand alloy composition dictionary into separate columns
    alloy_composition = curve_info.get('alloy_composition', {})
    for element, percentage in alloy_composition.items():
        # Clean element name (remove any units or extra info)
        element_clean = element.split('(')[0].strip() if '(' in element else element.strip()
        row[f'{element_clean}_pct'] = percentage
    
    df_data.append(row)

# Create DataFrame
df = pd.DataFrame(df_data)

print("="*80)
print("STEP 3: DataFrame Created")
print("="*80)
print(f"✓ DataFrame shape: {df.shape} (rows: {df.shape[0]}, columns: {df.shape[1]})")
print(f"✓ Total curves: {len(df)}")

# Show column categories
metadata_cols = ['paper_title', 'graph_id', 'curve_id', 'curve_label']
axis_cols = [c for c in df.columns if 'axis' in c.lower() or 'unit' in c.lower() or c.startswith('is_')]
km_cols = [c for c in df.columns if c.startswith('km_')]
alloy_cols = [c for c in df.columns if c.endswith('_pct')]
other_cols = ['curve_data']

print(f"\n📋 Column Categories:")
print(f"  Metadata columns: {len(metadata_cols)}")
print(f"  Axis/Unit columns: {len(axis_cols)}")
print(f"  Kocks-Mecking parameters: {len(km_cols)}")
print(f"  Alloy composition columns: {len(alloy_cols)}")
print(f"  Other: {len(other_cols)}")

print(f"\n📊  columns names:")
print(f"  Metadata: {metadata_cols}")
print(f"  Alloy elements found: {sorted([c.replace('_pct', '') for c in alloy_cols])}...")  # Show first 10
print(f"  KM parameters: {km_cols}")

print(f"\n📈 First few rows preview:")
print(df[metadata_cols + ['x_axis_label', 'y_axis_label', 'y_unit'] + km_cols + other_cols + alloy_cols].head())

print(f"\n✓ DataFrame ready! All data is now in tabular form.")
